In [ ]:
import sys
import os 

import numpy as np
import torch
%matplotlib inline
import matplotlib.pyplot as plt

from train_lstm_gan import TrainLSTMGAN

from lg_dataset import LGDataset

import json
import math

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import pandas as pd
from tensorflow import keras

In [ ]:
EVAL_BEST = False

In [ ]:
results = {}

for item in os.listdir('.'):
  if not os.path.isdir(item): continue
  if not item.endswith('_epoch'): continue
  epoch = int(item.split('_')[0])
  
  results[epoch] = {
    'fake_data': np.load(item + '/fake_data.npy'),
    'pca_fake': np.load(item + '/pca_fake.npy'),
    'pca_real': np.load(item + '/pca_real.npy'),
    'tsne_fake': np.load(item + '/tsne_fake.npy'),
    'tsne_real': np.load(item + '/tsne_real.npy')
  }
  
result_epochs = list(results.keys())
result_epochs.sort()

## Loss

In [ ]:
d_loss_path = 'd_loss.json'
g_loss_path = 'g_loss.json'

if EVAL_BEST:
  d_best_path = 'best_discriminator.json'
  g_best_path = 'best_generator.json'

with open(d_loss_path) as f:
  d_loss = json.load(f)
with open(g_loss_path) as f:
  g_loss = json.load(f)

if EVAL_BEST:
  with open(d_best_path) as f:
    d_best = json.load(f)
  with open(g_best_path) as f:
    g_best = json.load(f)
  
plt.plot(g_loss, label='generator loss')
plt.plot(d_loss, label='discriminator loss')

if EVAL_BEST:
  plt.scatter(g_best['epoch']-1, g_best['loss'], label='best generator')
  plt.scatter(d_best['epoch']-1, d_best['loss'], label='best discriminator')
plt.legend()
plt.show()

## Data Visualization

In [ ]:
real_dataset = LGDataset('../../data/lg_600_data.npy')
fig = plt.figure(figsize=(15,3), dpi=150)
ax = fig.add_subplot()
ax.plot(real_dataset.data[0][:300])
ax.set_title('Real data')

In [ ]:
fig = plt.figure(figsize=(15, 3*len(result_epochs)), dpi=150)
for i, epoch in enumerate(result_epochs):
  ax_fake_data = fig.add_subplot(len(result_epochs), 1, i+1)
  ax_fake_data.plot(results[epoch]['fake_data'][0])
  ax_fake_data.set_title(f'Synthetic data (epoch:{epoch})')

## Data Distribution

In [ ]:
fig = plt.figure(figsize=(15, 5*len(result_epochs)), dpi=150)
for i, epoch in enumerate(result_epochs):
  ax_pca = fig.add_subplot(len(result_epochs), 2, i*2+1)
  ax_pca.scatter(results[epoch]['pca_real'][:,0], results[epoch]['pca_real'][:,1], alpha=0.2, c='r', label='Real')
  ax_pca.scatter(results[epoch]['pca_fake'][:,0], results[epoch]['pca_fake'][:,1], alpha=0.2, c='b', label='Synthetic')
  ax_pca.legend()
  ax_pca.set_title(f'PCA (epoch:{epoch})')
  
  ax_tsne = fig.add_subplot(len(result_epochs), 2, i*2+2)
  ax_tsne.scatter(results[epoch]['tsne_real'][:,0], results[epoch]['tsne_real'][:,1], alpha=0.2, c='r', label='Real')
  ax_tsne.scatter(results[epoch]['tsne_fake'][:,0], results[epoch]['tsne_fake'][:,1], alpha=0.2, c='b', label='Synthetic')
  ax_tsne.legend()
  ax_tsne.set_title(f't-SNE (epoch:{epoch})')